# Loading everything

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from statistics import mean

In [2]:
batch1 = pickle.load(open(r'batch1.pkl', 'rb'))
#remove batteries that do not reach 80% capacity
del batch1['b1c8']
del batch1['b1c10']
del batch1['b1c12']
del batch1['b1c13']
del batch1['b1c22']

In [3]:
numBat1 = len(batch1.keys())
numBat1

41

In [4]:
batch2 = pickle.load(open(r'batch2.pkl','rb'))

In [5]:
# There are four cells from batch1 that carried into batch2, we'll remove the data from batch2
# and put it with the correct cell from batch1
batch2_keys = ['b2c7', 'b2c8', 'b2c9', 'b2c15', 'b2c16']
batch1_keys = ['b1c0', 'b1c1', 'b1c2', 'b1c3', 'b1c4']
add_len = [662, 981, 1060, 208, 482];

In [6]:
for i, bk in enumerate(batch1_keys):
    batch1[bk]['cycle_life'] = batch1[bk]['cycle_life'] + add_len[i]
    for j in batch1[bk]['summary'].keys():
        if j == 'cycle':
            batch1[bk]['summary'][j] = np.hstack((batch1[bk]['summary'][j], batch2[batch2_keys[i]]['summary'][j] + len(batch1[bk]['summary'][j])))
        else:
            batch1[bk]['summary'][j] = np.hstack((batch1[bk]['summary'][j], batch2[batch2_keys[i]]['summary'][j]))
    last_cycle = len(batch1[bk]['cycles'].keys())
    for j, jk in enumerate(batch2[batch2_keys[i]]['cycles'].keys()):
        batch1[bk]['cycles'][str(last_cycle + j)] = batch2[batch2_keys[i]]['cycles'][jk]

In [7]:
del batch2['b2c7']
del batch2['b2c8']
del batch2['b2c9']
del batch2['b2c15']
del batch2['b2c16']

In [8]:
numBat2 = len(batch2.keys())
numBat2

43

In [9]:
batch3 = pickle.load(open(r'batch3.pkl','rb'))
# remove noisy channels from batch3
del batch3['b3c37']
del batch3['b3c2']
del batch3['b3c23']
del batch3['b3c32']
del batch3['b3c42']
del batch3['b3c43']

In [10]:
numBat3 = len(batch3.keys())
numBat3

40

In [11]:
numBat = numBat1 + numBat2 + numBat3
numBat

124

In [12]:
bat_dict = {**batch1, **batch2, **batch3}

In [13]:
for i in bat_dict.keys():
    print(i)

b1c0
b1c1
b1c2
b1c3
b1c4
b1c5
b1c6
b1c7
b1c9
b1c11
b1c14
b1c15
b1c16
b1c17
b1c18
b1c19
b1c20
b1c21
b1c23
b1c24
b1c25
b1c26
b1c27
b1c28
b1c29
b1c30
b1c31
b1c32
b1c33
b1c34
b1c35
b1c36
b1c37
b1c38
b1c39
b1c40
b1c41
b1c42
b1c43
b1c44
b1c45
b2c0
b2c1
b2c2
b2c3
b2c4
b2c5
b2c6
b2c10
b2c11
b2c12
b2c13
b2c14
b2c17
b2c18
b2c19
b2c20
b2c21
b2c22
b2c23
b2c24
b2c25
b2c26
b2c27
b2c28
b2c29
b2c30
b2c31
b2c32
b2c33
b2c34
b2c35
b2c36
b2c37
b2c38
b2c39
b2c40
b2c41
b2c42
b2c43
b2c44
b2c45
b2c46
b2c47
b3c0
b3c1
b3c3
b3c4
b3c5
b3c6
b3c7
b3c8
b3c9
b3c10
b3c11
b3c12
b3c13
b3c14
b3c15
b3c16
b3c17
b3c18
b3c19
b3c20
b3c21
b3c22
b3c24
b3c25
b3c26
b3c27
b3c28
b3c29
b3c30
b3c31
b3c33
b3c34
b3c35
b3c36
b3c38
b3c39
b3c40
b3c41
b3c44
b3c45


# Getting all the features

In [14]:
from scipy.stats import skew, kurtosis
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np

In [15]:
features_df = pd.DataFrame()

In [16]:
minimum_dQ_100_10 = np.zeros(len(bat_dict.keys()))
variance_dQ_100_10 = np.zeros(len(bat_dict.keys()))
skewness_dQ_100_10 = np.zeros(len(bat_dict.keys()))
kurtosis_dQ_100_10 = np.zeros(len(bat_dict.keys()))

for i, cell in enumerate(bat_dict.values()):
    c10 = cell['cycles']['10']
    c100 = cell['cycles']['100']
    dQ_100_10 = c100['Qdlin'] - c10['Qdlin']
    
    minimum_dQ_100_10[i] = np.log(np.abs(np.min(dQ_100_10)))
    variance_dQ_100_10[i] = np.log(np.var(dQ_100_10))
    skewness_dQ_100_10[i] = np.log(np.abs(skew(dQ_100_10)))
    kurtosis_dQ_100_10[i] = np.log(np.abs(kurtosis(dQ_100_10)))

features_df["minimum_dQ_100_10"] = minimum_dQ_100_10
features_df["variance_dQ_100_10"] = variance_dQ_100_10
features_df["skewness_dQ_100_10"] = skewness_dQ_100_10
features_df["kurtosis_dQ_100_10"] = kurtosis_dQ_100_10


slope_lin_fit_2_100 = np.zeros(len(bat_dict.keys()))
intercept_lin_fit_2_100 = np.zeros(len(bat_dict.keys()))
discharge_capacity_2 = np.zeros(len(bat_dict.keys()))
diff_discharge_capacity_max_2 = np.zeros(len(bat_dict.keys()))

for i, cell in enumerate(bat_dict.values()):
    # Compute linear fit for cycles 2 to 100:
    q = cell['summary']['QD'][1:100].reshape(-1, 1)  # discharge cappacities; q.shape = (99, 1); 
    X = cycle_numbers = cell['summary']['cycle'][1:100].reshape(-1, 1)  # Cylce index from 2 to 100; X.shape = (99, 1)
    
    linear_regressor_2_100 = LinearRegression()
    linear_regressor_2_100.fit(X, q)
    
    slope_lin_fit_2_100[i] = linear_regressor_2_100.coef_[0]
    intercept_lin_fit_2_100[i] = linear_regressor_2_100.intercept_
    discharge_capacity_2[i] = q[0][0]
    diff_discharge_capacity_max_2[i] = np.max(q) - q[0][0]
    
features_df["slope_lin_fit_2_100"] = slope_lin_fit_2_100
features_df["intercept_lin_fit_2_100"] = intercept_lin_fit_2_100
features_df["discharge_capacity_2"] = discharge_capacity_2
features_df["diff_discharge_capacity_max_2"] = diff_discharge_capacity_max_2

mean_charge_time = np.zeros(len(bat_dict.keys()))
minimum_IR = np.zeros(len(bat_dict.keys()))
diff_IR_100_2 = np.zeros(len(bat_dict.keys()))


for i, cell in enumerate(bat_dict.values()):
    mean_charge_time[i] = np.mean(cell['summary']['chargetime'][1:6])
    minimum_IR[i] = np.min(cell['summary']['IR'][1:100])
    diff_IR_100_2[i] = cell['summary']['IR'][100] - cell['summary']['IR'][1]
    
features_df["mean_charge_time"] = mean_charge_time
features_df["minimum_IR"] = minimum_IR
features_df["diff_IR_100_2"] = diff_IR_100_2

In [17]:
features_df.head()

,minimum_dQ_100_10,variance_dQ_100_10,skewness_dQ_100_10,kurtosis_dQ_100_10,slope_lin_fit_2_100,intercept_lin_fit_2_100,discharge_capacity_2,diff_discharge_capacity_max_2,mean_charge_time,minimum_IR,diff_IR_100_2
0,-4.720901,-11.631570,-0.319238,-0.094314,-0.000207,1.091144,1.070689,0.468365,13.374894,0.016444,-0.000092
1,-4.709132,-11.825562,-1.148991,0.037823,0.000006,1.080965,1.075301,0.009288,13.409150,0.000000,-0.000034
2,-4.575223,-11.402131,-0.826717,0.160194,0.000010,1.084846,1.079922,0.008131,13.358242,0.000000,-0.000009
3,-3.922042,-10.099939,-0.741646,0.117089,0.000017,1.084075,1.079723,0.006530,12.025140,0.000000,0.000044
4,-4.230762,-10.602896,-1.052154,0.308340,0.000019,1.081875,1.078417,0.005900,12.041851,0.000000,-0.000036


In [18]:
features_df.shape

(124, 11)

In [19]:
cols = features_df.columns

# Splitting the data 80-20

In [20]:
target_df = pd.DataFrame()
cycle_life = []
for i in bat_dict:
    cycle_life.append(np.float(bat_dict[i]["cycle_life"]))
cycle_life = pd.Series(cycle_life)
target_df["cycle_life"] = cycle_life
target_df.head()

,cycle_life
0,1852.0
1,2160.0
2,2237.0
3,1434.0
4,1709.0


In [21]:
x = features_df.head(100)
y = cycle_life.head(100)

In [22]:
import numpy as np
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=False)

In [23]:
x_train

,minimum_dQ_100_10,variance_dQ_100_10,skewness_dQ_100_10,kurtosis_dQ_100_10,slope_lin_fit_2_100,intercept_lin_fit_2_100,discharge_capacity_2,diff_discharge_capacity_max_2,mean_charge_time,minimum_IR,diff_IR_100_2
0,-4.720901,-11.631570,-0.319238,-0.094314,-0.000207,1.091144,1.070689,0.468365,13.374894,0.016444,-0.000092
1,-4.709132,-11.825562,-1.148991,0.037823,0.000006,1.080965,1.075301,0.009288,13.409150,0.000000,-0.000034
2,-4.575223,-11.402131,-0.826717,0.160194,0.000010,1.084846,1.079922,0.008131,13.358242,0.000000,-0.000009
3,-3.922042,-10.099939,-0.741646,0.117089,0.000017,1.084075,1.079723,0.006530,12.025140,0.000000,0.000044
4,-4.230762,-10.602896,-1.052154,0.308340,0.000019,1.081875,1.078417,0.005900,12.041851,0.000000,-0.000036
...,...,...,...,...,...,...,...,...,...,...,...
75,-2.889263,-7.948993,-5.117145,0.296855,0.000032,1.084284,1.078461,0.008421,10.144479,0.017882,0.000002
76,-2.966849,-8.134920,-2.733681,0.207714,-0.000020,1.082269,1.076552,0.006568,10.131583,0.016944,-0.000549
77,-3.103094,-8.484278,-1.505743,0.122695,0.000017,1.073883,1.068205,0.007904,10.128036,0.017491,-0.000199
78,-2.775922,-8.131572,-2.799686,0.217362,0.000012,1.082518,1.078024,0.006087,10.130454,0.016998,0.000038


In [24]:
x_test

,minimum_dQ_100_10,variance_dQ_100_10,skewness_dQ_100_10,kurtosis_dQ_100_10,slope_lin_fit_2_100,intercept_lin_fit_2_100,discharge_capacity_2,diff_discharge_capacity_max_2,mean_charge_time,minimum_IR,diff_IR_100_2
80,-2.862830,-7.962522,-4.905592,0.227158,2.227313e-05,1.075214,1.070887,0.006366,10.146905,0.017409,0.000223
81,-3.007304,-8.233459,-2.076041,0.256923,-7.060437e-06,1.074868,1.070473,0.005186,10.097114,0.016838,-0.000127
82,-2.755650,-7.722303,-2.924819,0.231593,2.937986e-05,1.077632,1.072614,0.007384,10.146917,0.017552,0.000202
83,-3.556632,-9.373764,-1.171876,0.146449,-9.553368e-06,1.049389,1.042137,0.008113,10.112717,0.017026,-0.000819
84,-3.746697,-9.746089,-1.677966,0.170878,-5.346195e-06,1.070659,1.067455,0.003916,10.044531,0.015180,-0.000126
85,-3.770055,-9.782774,-1.334613,0.136729,-1.471934e-05,1.068472,1.065060,0.003680,10.041139,0.014932,-0.000108
86,-3.606255,-9.485617,-0.765798,0.018072,-1.265485e-05,1.065298,1.062889,0.002645,10.042065,0.014885,0.000042
87,-3.708104,-9.650334,-0.918144,0.111117,-5.233105e-06,1.074805,1.072477,0.002788,10.045031,0.015085,-0.000065
88,-3.638100,-9.432684,-1.873386,0.250943,-1.602090e-05,1.053974,1.051451,0.002605,10.041910,0.015131,-0.000158
89,-2.920147,-8.096790,-0.747359,-0.134090,-2.023978e-04,1.064082,1.062612,0.000459,10.062472,0.013527,0.000325


# APPROACH 1

## VARIANCE

In [25]:
error_list = []
real_lifetime_list = []
predicted_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    df["cell_test"] = pd.Series()
    df["cell_train"] = pd.Series()
    df["distance"] = pd.Series()
    cell_test = []
    cell_train = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test.append(x_test["variance_dQ_100_10"][i])
    for a in x_train.index:
        cell_train.append(x_train["variance_dQ_100_10"][a])
    df["cell_test"] = pd.Series(cell_test)
    df["cell_train"] = pd.Series(cell_train)
    df["distance"] = abs(df["cell_test"] - df["cell_train"])

    #finding the most similar cell
    minimum = min(df["distance"])
    
    #calculating the error
    real_lifetime = y_test[i]
    real_lifetime_list.append(float(real_lifetime))

    predicted_lifetime = y_train[df.loc[df['distance'] == minimum].index]
    predicted_lifetime_list.append(float(predicted_lifetime))

    error = float(abs(real_lifetime - predicted_lifetime))
    error = error/real_lifetime
    error_list.append(error*100)

    #print
    print(df)
    print("Error: ", np.array(error))

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipyk

    cell_test  cell_train  distance
0   -7.962522  -11.631570  3.669048
1   -7.962522  -11.825562  3.863040
2   -7.962522  -11.402131  3.439609
3   -7.962522  -10.099939  2.137417
4   -7.962522  -10.602896  2.640374
..        ...         ...       ...
75  -7.962522   -7.948993  0.013529
76  -7.962522   -8.134920  0.172398
77  -7.962522   -8.484278  0.521756
78  -7.962522   -8.131572  0.169050
79  -7.962522   -7.650151  0.312371

[80 rows x 3 columns]
Error:  0.08096280087527352
    cell_test  cell_train  distance
0   -8.233459  -11.631570  3.398111
1   -8.233459  -11.825562  3.592103
2   -8.233459  -11.402131  3.168672
3   -8.233459  -10.099939  1.866480
4   -8.233459  -10.602896  2.369437
..        ...         ...       ...
75  -8.233459   -7.948993  0.284466
76  -8.233459   -8.134920  0.098539
77  -8.233459   -8.484278  0.250819
78  -8.233459   -8.131572  0.101887
79  -8.233459   -7.650151  0.583308

[80 rows x 3 columns]
Error:  0.045174537987679675
    cell_test  cell_train  distan

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.


In [26]:
pd.Series(error_list).describe()

count    20.000000
mean     14.552456
std      12.318572
min       1.034807
25%       4.471729
50%      11.553093
75%      22.094214
max      47.405330
dtype: float64

## DISCHARGE

In [27]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #skewness
    df["cell_test_skewness"] = pd.Series()
    df["cell_train_skewness"] = pd.Series()
    #kurtosis
    df["cell_test_kurtosis"] = pd.Series()
    df["cell_train_kurtosis"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #diff discharge capacity max 2
    df["cell_test_ddc2"] = pd.Series()
    df["cell_train_ddc2"] = pd.Series()
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_skewness = []
    cell_test_kurtosis = []
    cell_test_dc2 = []
    cell_test_ddc2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_skewness = []
    cell_train_kurtosis = []
    cell_train_dc2 = []
    cell_train_ddc2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_skewness.append(x_test["skewness_dQ_100_10"][i])
        cell_test_kurtosis.append(x_test["kurtosis_dQ_100_10"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_ddc2.append(x_test["diff_discharge_capacity_max_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_skewness.append(x_train["skewness_dQ_100_10"][a])
        cell_train_kurtosis.append(x_train["kurtosis_dQ_100_10"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_ddc2.append(x_train["diff_discharge_capacity_max_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_skewness"] = pd.Series(cell_test_skewness)
    df["cell_test_kurtosis"] = pd.Series(cell_test_kurtosis)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_ddc2"] = pd.Series(cell_test_ddc2)

    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_skewness"] = pd.Series(cell_train_skewness)
    df["cell_train_kurtosis"] = pd.Series(cell_train_kurtosis)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_ddc2"] = pd.Series(cell_train_ddc2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_skewness"] - df["cell_train_skewness"])**2
    d = (df["cell_test_kurtosis"] - df["cell_train_kurtosis"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_ddc2"] - df["cell_train_ddc2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f)

    #finding the most similar cell
    minimum = min(df["distance"])
    
    #calculating the error
    real_lifetime = y_test[i]
    real_lifetime_list.append(float(real_lifetime))

    predicted_lifetime = y_train[df.loc[df['distance'] == minimum].index]
    predicted_lifetime_list.append(float(predicted_lifetime))

    error = float(abs(real_lifetime - predicted_lifetime))
    error = error/real_lifetime
    error_list.append(error*100)

    #print
    print(df)
    print("Error: ", np.array(error))

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -7.962522           -11.631570           -2.86283   
1            -7.962522           -11.825562           -2.86283   
2            -7.962522           -11.402131           -2.86283   
3            -7.962522           -10.099939           -2.86283   
4            -7.962522           -10.602896           -2.86283   
..                 ...                  ...                ...   
75           -7.962522            -7.948993           -2.86283   
76           -7.962522            -8.134920           -2.86283   
77           -7.962522            -8.484278           -2.86283   
78           -7.962522            -8.131572           -2.86283   
79           -7.962522            -7.650151           -2.86283   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901           -4.905592            -0.319238   
1            -4.709132           -4.905592            -1.148991   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: The default dtype for empty Series will be

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.432684           -11.631570            -3.6381   
1            -9.432684           -11.825562            -3.6381   
2            -9.432684           -11.402131            -3.6381   
3            -9.432684           -10.099939            -3.6381   
4            -9.432684           -10.602896            -3.6381   
..                 ...                  ...                ...   
75           -9.432684            -7.948993            -3.6381   
76           -9.432684            -8.134920            -3.6381   
77           -9.432684            -8.484278            -3.6381   
78           -9.432684            -8.131572            -3.6381   
79           -9.432684            -7.650151            -3.6381   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901           -1.873386            -0.319238   
1            -4.709132           -1.873386            -1.148991   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: The default dtype for empty Series will be

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.071859           -11.631570          -3.442255   
1            -9.071859           -11.825562          -3.442255   
2            -9.071859           -11.402131          -3.442255   
3            -9.071859           -10.099939          -3.442255   
4            -9.071859           -10.602896          -3.442255   
..                 ...                  ...                ...   
75           -9.071859            -7.948993          -3.442255   
76           -9.071859            -8.134920          -3.442255   
77           -9.071859            -8.484278          -3.442255   
78           -9.071859            -8.131572          -3.442255   
79           -9.071859            -7.650151          -3.442255   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901            -5.03744            -0.319238   
1            -4.709132            -5.03744            -1.148991   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

In [28]:
pd.Series(error_list).describe()

count    40.000000
mean     17.941623
std      15.529444
min       0.437637
25%       6.039246
50%      14.869719
75%      25.443811
max      58.534323
dtype: float64

## FULL MODEL

In [29]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #slope_lin_fit_2_100
    df["cell_test_slope_lin_fit_2_100"] = pd.Series()
    df["cell_train_slope_lin_fit_2_100"] = pd.Series()
    #intercept_lin_fit_2_100
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series()
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #mean charge time
    df["cell_test_mean_charge_time"] = pd.Series()
    df["cell_train_mean_charge_time"] = pd.Series()  
    #minimum IR
    df["cell_test_minimum_IR"] = pd.Series()
    df["cell_train_minimum_IR"] = pd.Series()
    #diff_IR_100_2
    df["cell_test_diff_IR_100_2"] = pd.Series()
    df["cell_train_diff_IR_100_2"] = pd.Series()  
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_slope_lin_fit_2_100 = []
    cell_test_intercept_lin_fit_2_100 = []
    cell_test_dc2 = []
    cell_test_mean_charge_time = []
    cell_test_minimum_IR = []
    cell_test_diff_IR_100_2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_slope_lin_fit_2_100 = []
    cell_train_intercept_lin_fit_2_100 = []
    cell_train_dc2 = []
    cell_train_mean_charge_time = []
    cell_train_minimum_IR = []
    cell_train_diff_IR_100_2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_slope_lin_fit_2_100.append(x_test["slope_lin_fit_2_100"][i])
        cell_test_intercept_lin_fit_2_100.append(x_test["intercept_lin_fit_2_100"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_mean_charge_time.append(x_test["mean_charge_time"][i])
        cell_test_minimum_IR.append(x_test["minimum_IR"][i])
        cell_test_diff_IR_100_2.append(x_test["diff_IR_100_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_slope_lin_fit_2_100.append(x_train["slope_lin_fit_2_100"][a])
        cell_train_intercept_lin_fit_2_100.append(x_train["intercept_lin_fit_2_100"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_mean_charge_time.append(x_train["mean_charge_time"][a])
        cell_train_minimum_IR.append(x_train["minimum_IR"][a])
        cell_train_diff_IR_100_2.append(x_train["diff_IR_100_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_slope_lin_fit_2_100"] = pd.Series(cell_test_slope_lin_fit_2_100)
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series(cell_test_intercept_lin_fit_2_100)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_mean_charge_time"] = pd.Series(cell_test_mean_charge_time)
    df["cell_test_minimum_IR"] = pd.Series(cell_test_minimum_IR)
    df["cell_test_diff_IR_100_2"] = pd.Series(cell_test_diff_IR_100_2)


    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_slope_lin_fit_2_100"] = pd.Series(cell_train_slope_lin_fit_2_100)
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series(cell_train_intercept_lin_fit_2_100)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_mean_charge_time"] = pd.Series(cell_train_mean_charge_time)
    df["cell_train_minimum_IR"] = pd.Series(cell_train_minimum_IR)
    df["cell_train_diff_IR_100_2"] = pd.Series(cell_train_diff_IR_100_2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_slope_lin_fit_2_100"] - df["cell_train_slope_lin_fit_2_100"])**2
    d = (df["cell_test_intercept_lin_fit_2_100"] - df["cell_train_intercept_lin_fit_2_100"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_mean_charge_time"] - df["cell_train_mean_charge_time"])**2
    g = (df["cell_test_minimum_IR"] - df["cell_train_minimum_IR"])**2
    h = (df["cell_test_diff_IR_100_2"] - df["cell_train_diff_IR_100_2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f+g+h)

    #finding the most similar cell
    minimum = min(df["distance"])
    
    #calculating the error
    real_lifetime = y_test[i]
    real_lifetime_list.append(float(real_lifetime))

    predicted_lifetime = y_train[df.loc[df['distance'] == minimum].index]
    predicted_lifetime_list.append(float(predicted_lifetime))

    error = float(abs(real_lifetime - predicted_lifetime))
    error = error/real_lifetime
    error_list.append(error*100)

    #print
    print(df)
    print("Error: ", np.array(error))


/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -7.962522           -11.631570           -2.86283   
1            -7.962522           -11.825562           -2.86283   
2            -7.962522           -11.402131           -2.86283   
3            -7.962522           -10.099939           -2.86283   
4            -7.962522           -10.602896           -2.86283   
..                 ...                  ...                ...   
75           -7.962522            -7.948993           -2.86283   
76           -7.962522            -8.134920           -2.86283   
77           -7.962522            -8.484278           -2.86283   
78           -7.962522            -8.131572           -2.86283   
79           -7.962522            -7.650151           -2.86283   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                       0.000022   
1            -4.709132                       0.000022   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.782774           -11.631570          -3.770055   
1            -9.782774           -11.825562          -3.770055   
2            -9.782774           -11.402131          -3.770055   
3            -9.782774           -10.099939          -3.770055   
4            -9.782774           -10.602896          -3.770055   
..                 ...                  ...                ...   
75           -9.782774            -7.948993          -3.770055   
76           -9.782774            -8.134920          -3.770055   
77           -9.782774            -8.484278          -3.770055   
78           -9.782774            -8.131572          -3.770055   
79           -9.782774            -7.650151          -3.770055   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                      -0.000015   
1            -4.709132                      -0.000015   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0             -8.09679           -11.631570          -2.920147   
1             -8.09679           -11.825562          -2.920147   
2             -8.09679           -11.402131          -2.920147   
3             -8.09679           -10.099939          -2.920147   
4             -8.09679           -10.602896          -2.920147   
..                 ...                  ...                ...   
75            -8.09679            -7.948993          -2.920147   
76            -8.09679            -8.134920          -2.920147   
77            -8.09679            -8.484278          -2.920147   
78            -8.09679            -8.131572          -2.920147   
79            -8.09679            -7.650151          -2.920147   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                      -0.000202   
1            -4.709132                      -0.000202   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy


Error:  0.0037105751391465678
    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -8.972545           -11.631570          -3.388775   
1            -8.972545           -11.825562          -3.388775   
2            -8.972545           -11.402131          -3.388775   
3            -8.972545           -10.099939          -3.388775   
4            -8.972545           -10.602896          -3.388775   
..                 ...                  ...                ...   
75           -8.972545            -7.948993          -3.388775   
76           -8.972545            -8.134920          -3.388775   
77           -8.972545            -8.484278          -3.388775   
78           -8.972545            -8.131572          -3.388775   
79           -8.972545            -7.650151          -3.388775   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                      -0.000018   
1            -4.709132                      -0.000018   
2    

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: The default dtype for empty Series will be

In [30]:
pd.Series(error_list).describe()

count    60.000000
mean     15.885028
std      14.358526
min       0.116550
25%       4.471729
50%      13.286713
75%      22.690583
max      58.534323
dtype: float64

# APPROACH 2 (3 most similar cells)

## VARIANCE

In [56]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    df["cell_test"] = pd.Series()
    df["cell_train"] = pd.Series()
    df["distance"] = pd.Series()
    cell_test = []
    cell_train = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test.append(x_test["variance_dQ_100_10"][i])
    for a in x_train.index:
        cell_train.append(x_train["variance_dQ_100_10"][a])
    df["cell_test"] = pd.Series(cell_test)
    df["cell_train"] = pd.Series(cell_train)
    df["distance"] = abs(df["cell_test"] - df["cell_train"])

    #finding the 3 most similar cells
    three_most_similar_cells = df.nsmallest(3, 'distance')
    number1 = int(y_train[three_most_similar_cells.iloc[0].name])
    number2 = int(y_train[three_most_similar_cells.iloc[1].name])
    number3 = int(y_train[three_most_similar_cells.iloc[2].name])

    #getting max and min
    list_of_three_most_similar_cells = [number1, number2, number3]
    max_predicted_lifetime = max(list_of_three_most_similar_cells)
    min_predicted_lifetime = min(list_of_three_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))

    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")
    
    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)


    cell_test  cell_train  distance
0   -7.962522  -11.631570  3.669048
1   -7.962522  -11.825562  3.863040
2   -7.962522  -11.402131  3.439609
3   -7.962522  -10.099939  2.137417
4   -7.962522  -10.602896  2.640374
..        ...         ...       ...
75  -7.962522   -7.948993  0.013529
76  -7.962522   -8.134920  0.172398
77  -7.962522   -8.484278  0.521756
78  -7.962522   -8.131572  0.169050
79  -7.962522   -7.650151  0.312371

[80 rows x 3 columns]
Predicted lifetime range is from:  459 to  527
Actual lifetime:  457


    cell_test  cell_train  distance
0   -8.233459  -11.631570  3.398111
1   -8.233459  -11.825562  3.592103
2   -8.233459  -11.402131  3.168672
3   -8.233459  -10.099939  1.866480
4   -8.233459  -10.602896  2.369437
..        ...         ...       ...
75  -8.233459   -7.948993  0.284466
76  -8.233459   -8.134920  0.098539
77  -8.233459   -8.484278  0.250819
78  -8.233459   -8.131572  0.101887
79  -8.233459   -7.650151  0.583308

[80 rows x 3 columns]
Predicted lifetime 

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/e

In [59]:
accuracy_list.count("No")

6

In [60]:
mean(range_list)

278.05

## DISCHARGE

In [34]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #skewness
    df["cell_test_skewness"] = pd.Series()
    df["cell_train_skewness"] = pd.Series()
    #kurtosis
    df["cell_test_kurtosis"] = pd.Series()
    df["cell_train_kurtosis"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #diff discharge capacity max 2
    df["cell_test_ddc2"] = pd.Series()
    df["cell_train_ddc2"] = pd.Series()
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_skewness = []
    cell_test_kurtosis = []
    cell_test_dc2 = []
    cell_test_ddc2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_skewness = []
    cell_train_kurtosis = []
    cell_train_dc2 = []
    cell_train_ddc2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_skewness.append(x_test["skewness_dQ_100_10"][i])
        cell_test_kurtosis.append(x_test["kurtosis_dQ_100_10"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_ddc2.append(x_test["diff_discharge_capacity_max_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_skewness.append(x_train["skewness_dQ_100_10"][a])
        cell_train_kurtosis.append(x_train["kurtosis_dQ_100_10"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_ddc2.append(x_train["diff_discharge_capacity_max_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_skewness"] = pd.Series(cell_test_skewness)
    df["cell_test_kurtosis"] = pd.Series(cell_test_kurtosis)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_ddc2"] = pd.Series(cell_test_ddc2)

    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_skewness"] = pd.Series(cell_train_skewness)
    df["cell_train_kurtosis"] = pd.Series(cell_train_kurtosis)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_ddc2"] = pd.Series(cell_train_ddc2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_skewness"] - df["cell_train_skewness"])**2
    d = (df["cell_test_kurtosis"] - df["cell_train_kurtosis"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_ddc2"] - df["cell_train_ddc2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f)

    #finding the 3 most similar cells
    three_most_similar_cells = df.nsmallest(3, 'distance')
    number1 = int(y_train[three_most_similar_cells.iloc[0].name])
    number2 = int(y_train[three_most_similar_cells.iloc[1].name])
    number3 = int(y_train[three_most_similar_cells.iloc[2].name])

    #getting max and min
    list_of_three_most_similar_cells = [number1, number2, number3]
    max_predicted_lifetime = max(list_of_three_most_similar_cells)
    min_predicted_lifetime = min(list_of_three_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

    
        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")

    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)


/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -7.962522           -11.631570           -2.86283   
1            -7.962522           -11.825562           -2.86283   
2            -7.962522           -11.402131           -2.86283   
3            -7.962522           -10.099939           -2.86283   
4            -7.962522           -10.602896           -2.86283   
..                 ...                  ...                ...   
75           -7.962522            -7.948993           -2.86283   
76           -7.962522            -8.134920           -2.86283   
77           -7.962522            -8.484278           -2.86283   
78           -7.962522            -8.131572           -2.86283   
79           -7.962522            -7.650151           -2.86283   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901           -4.905592            -0.319238   
1            -4.709132           -4.905592            -1.148991   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: The default dtype for empty Series will be

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.432684           -11.631570            -3.6381   
1            -9.432684           -11.825562            -3.6381   
2            -9.432684           -11.402131            -3.6381   
3            -9.432684           -10.099939            -3.6381   
4            -9.432684           -10.602896            -3.6381   
..                 ...                  ...                ...   
75           -9.432684            -7.948993            -3.6381   
76           -9.432684            -8.134920            -3.6381   
77           -9.432684            -8.484278            -3.6381   
78           -9.432684            -8.131572            -3.6381   
79           -9.432684            -7.650151            -3.6381   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901           -1.873386            -0.319238   
1            -4.709132           -1.873386            -1.148991   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

In [35]:
accuracy_list.count("No")

12

In [36]:
mean(range_list)

319.9

## FULL MODEL

In [37]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #slope_lin_fit_2_100
    df["cell_test_slope_lin_fit_2_100"] = pd.Series()
    df["cell_train_slope_lin_fit_2_100"] = pd.Series()
    #intercept_lin_fit_2_100
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series()
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #mean charge time
    df["cell_test_mean_charge_time"] = pd.Series()
    df["cell_train_mean_charge_time"] = pd.Series()  
    #minimum IR
    df["cell_test_minimum_IR"] = pd.Series()
    df["cell_train_minimum_IR"] = pd.Series()
    #diff_IR_100_2
    df["cell_test_diff_IR_100_2"] = pd.Series()
    df["cell_train_diff_IR_100_2"] = pd.Series()  
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_slope_lin_fit_2_100 = []
    cell_test_intercept_lin_fit_2_100 = []
    cell_test_dc2 = []
    cell_test_mean_charge_time = []
    cell_test_minimum_IR = []
    cell_test_diff_IR_100_2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_slope_lin_fit_2_100 = []
    cell_train_intercept_lin_fit_2_100 = []
    cell_train_dc2 = []
    cell_train_mean_charge_time = []
    cell_train_minimum_IR = []
    cell_train_diff_IR_100_2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_slope_lin_fit_2_100.append(x_test["slope_lin_fit_2_100"][i])
        cell_test_intercept_lin_fit_2_100.append(x_test["intercept_lin_fit_2_100"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_mean_charge_time.append(x_test["mean_charge_time"][i])
        cell_test_minimum_IR.append(x_test["minimum_IR"][i])
        cell_test_diff_IR_100_2.append(x_test["diff_IR_100_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_slope_lin_fit_2_100.append(x_train["slope_lin_fit_2_100"][a])
        cell_train_intercept_lin_fit_2_100.append(x_train["intercept_lin_fit_2_100"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_mean_charge_time.append(x_train["mean_charge_time"][a])
        cell_train_minimum_IR.append(x_train["minimum_IR"][a])
        cell_train_diff_IR_100_2.append(x_train["diff_IR_100_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_slope_lin_fit_2_100"] = pd.Series(cell_test_slope_lin_fit_2_100)
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series(cell_test_intercept_lin_fit_2_100)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_mean_charge_time"] = pd.Series(cell_test_mean_charge_time)
    df["cell_test_minimum_IR"] = pd.Series(cell_test_minimum_IR)
    df["cell_test_diff_IR_100_2"] = pd.Series(cell_test_diff_IR_100_2)


    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_slope_lin_fit_2_100"] = pd.Series(cell_train_slope_lin_fit_2_100)
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series(cell_train_intercept_lin_fit_2_100)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_mean_charge_time"] = pd.Series(cell_train_mean_charge_time)
    df["cell_train_minimum_IR"] = pd.Series(cell_train_minimum_IR)
    df["cell_train_diff_IR_100_2"] = pd.Series(cell_train_diff_IR_100_2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_slope_lin_fit_2_100"] - df["cell_train_slope_lin_fit_2_100"])**2
    d = (df["cell_test_intercept_lin_fit_2_100"] - df["cell_train_intercept_lin_fit_2_100"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_mean_charge_time"] - df["cell_train_mean_charge_time"])**2
    g = (df["cell_test_minimum_IR"] - df["cell_train_minimum_IR"])**2
    h = (df["cell_test_diff_IR_100_2"] - df["cell_train_diff_IR_100_2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f+g+h)

    #finding the 3 most similar cells
    three_most_similar_cells = df.nsmallest(3, 'distance')
    number1 = int(y_train[three_most_similar_cells.iloc[0].name])
    number2 = int(y_train[three_most_similar_cells.iloc[1].name])
    number3 = int(y_train[three_most_similar_cells.iloc[2].name])

    #getting max and min
    list_of_three_most_similar_cells = [number1, number2, number3]
    max_predicted_lifetime = max(list_of_three_most_similar_cells)
    min_predicted_lifetime = min(list_of_three_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")

    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)


/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -7.962522           -11.631570           -2.86283   
1            -7.962522           -11.825562           -2.86283   
2            -7.962522           -11.402131           -2.86283   
3            -7.962522           -10.099939           -2.86283   
4            -7.962522           -10.602896           -2.86283   
..                 ...                  ...                ...   
75           -7.962522            -7.948993           -2.86283   
76           -7.962522            -8.134920           -2.86283   
77           -7.962522            -8.484278           -2.86283   
78           -7.962522            -8.131572           -2.86283   
79           -7.962522            -7.650151           -2.86283   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                       0.000022   
1            -4.709132                       0.000022   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning


Predicted lifetime range is from:  857 to  880
Actual lifetime:  1063


    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.485617           -11.631570          -3.606255   
1            -9.485617           -11.825562          -3.606255   
2            -9.485617           -11.402131          -3.606255   
3            -9.485617           -10.099939          -3.606255   
4            -9.485617           -10.602896          -3.606255   
..                 ...                  ...                ...   
75           -9.485617            -7.948993          -3.606255   
76           -9.485617            -8.134920          -3.606255   
77           -9.485617            -8.484278          -3.606255   
78           -9.485617            -8.131572          -3.606255   
79           -9.485617            -7.650151          -3.606255   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                      -0.000013   
1            -4.7091

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:35: DeprecationWarning: The default dtype for empty Series will be

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0           -10.332671           -11.631570          -4.066422   
1           -10.332671           -11.825562          -4.066422   
2           -10.332671           -11.402131          -4.066422   
3           -10.332671           -10.099939          -4.066422   
4           -10.332671           -10.602896          -4.066422   
..                 ...                  ...                ...   
75          -10.332671            -7.948993          -4.066422   
76          -10.332671            -8.134920          -4.066422   
77          -10.332671            -8.484278          -4.066422   
78          -10.332671            -8.131572          -4.066422   
79          -10.332671            -7.650151          -4.066422   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                      -0.000002   
1            -4.709132                      -0.000002   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.071859           -11.631570          -3.442255   
1            -9.071859           -11.825562          -3.442255   
2            -9.071859           -11.402131          -3.442255   
3            -9.071859           -10.099939          -3.442255   
4            -9.071859           -10.602896          -3.442255   
..                 ...                  ...                ...   
75           -9.071859            -7.948993          -3.442255   
76           -9.071859            -8.134920          -3.442255   
77           -9.071859            -8.484278          -3.442255   
78           -9.071859            -8.131572          -3.442255   
79           -9.071859            -7.650151          -3.442255   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                      -0.000007   
1            -4.709132                      -0.000007   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

In [38]:
accuracy_list.count("No")

15

In [39]:
mean(range_list)

141.25

# APPROACH 3 (5 most similar cells)

## VARIANCE

In [40]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    df["cell_test"] = pd.Series()
    df["cell_train"] = pd.Series()
    df["distance"] = pd.Series()
    cell_test = []
    cell_train = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test.append(x_test["variance_dQ_100_10"][i])
    for a in x_train.index:
        cell_train.append(x_train["variance_dQ_100_10"][a])
    df["cell_test"] = pd.Series(cell_test)
    df["cell_train"] = pd.Series(cell_train)
    df["distance"] = abs(df["cell_test"] - df["cell_train"])

    #finding the most similar cells
    most_similar_cells = df.nsmallest(5, 'distance')
    number1 = int(y_train[most_similar_cells.iloc[0].name])
    number2 = int(y_train[most_similar_cells.iloc[1].name])
    number3 = int(y_train[most_similar_cells.iloc[2].name])
    number4 = int(y_train[most_similar_cells.iloc[3].name])
    number5 = int(y_train[most_similar_cells.iloc[4].name])

    #getting max and min
    list_of_most_similar_cells = [number1, number2, number3, number4, number5]
    max_predicted_lifetime = max(list_of_most_similar_cells)
    min_predicted_lifetime = min(list_of_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")
    
    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)


/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/e

    cell_test  cell_train  distance
0   -7.962522  -11.631570  3.669048
1   -7.962522  -11.825562  3.863040
2   -7.962522  -11.402131  3.439609
3   -7.962522  -10.099939  2.137417
4   -7.962522  -10.602896  2.640374
..        ...         ...       ...
75  -7.962522   -7.948993  0.013529
76  -7.962522   -8.134920  0.172398
77  -7.962522   -8.484278  0.521756
78  -7.962522   -8.131572  0.169050
79  -7.962522   -7.650151  0.312371

[80 rows x 3 columns]
Predicted lifetime range is from:  444 to  527
Actual lifetime:  457


    cell_test  cell_train  distance
0   -8.233459  -11.631570  3.398111
1   -8.233459  -11.825562  3.592103
2   -8.233459  -11.402131  3.168672
3   -8.233459  -10.099939  1.866480
4   -8.233459  -10.602896  2.369437
..        ...         ...       ...
75  -8.233459   -7.948993  0.284466
76  -8.233459   -8.134920  0.098539
77  -8.233459   -8.484278  0.250819
78  -8.233459   -8.131572  0.101887
79  -8.233459   -7.650151  0.583308

[80 rows x 3 columns]
Predicted lifetime 

In [41]:
accuracy_list.count("No")

5

In [42]:
mean(range_list)

377.5

## DISCHARGE

In [43]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #skewness
    df["cell_test_skewness"] = pd.Series()
    df["cell_train_skewness"] = pd.Series()
    #kurtosis
    df["cell_test_kurtosis"] = pd.Series()
    df["cell_train_kurtosis"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #diff discharge capacity max 2
    df["cell_test_ddc2"] = pd.Series()
    df["cell_train_ddc2"] = pd.Series()
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_skewness = []
    cell_test_kurtosis = []
    cell_test_dc2 = []
    cell_test_ddc2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_skewness = []
    cell_train_kurtosis = []
    cell_train_dc2 = []
    cell_train_ddc2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_skewness.append(x_test["skewness_dQ_100_10"][i])
        cell_test_kurtosis.append(x_test["kurtosis_dQ_100_10"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_ddc2.append(x_test["diff_discharge_capacity_max_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_skewness.append(x_train["skewness_dQ_100_10"][a])
        cell_train_kurtosis.append(x_train["kurtosis_dQ_100_10"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_ddc2.append(x_train["diff_discharge_capacity_max_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_skewness"] = pd.Series(cell_test_skewness)
    df["cell_test_kurtosis"] = pd.Series(cell_test_kurtosis)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_ddc2"] = pd.Series(cell_test_ddc2)

    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_skewness"] = pd.Series(cell_train_skewness)
    df["cell_train_kurtosis"] = pd.Series(cell_train_kurtosis)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_ddc2"] = pd.Series(cell_train_ddc2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_skewness"] - df["cell_train_skewness"])**2
    d = (df["cell_test_kurtosis"] - df["cell_train_kurtosis"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_ddc2"] - df["cell_train_ddc2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f)

    #finding the most similar cells
    most_similar_cells = df.nsmallest(5, 'distance')
    number1 = int(y_train[most_similar_cells.iloc[0].name])
    number2 = int(y_train[most_similar_cells.iloc[1].name])
    number3 = int(y_train[most_similar_cells.iloc[2].name])
    number4 = int(y_train[most_similar_cells.iloc[3].name])
    number5 = int(y_train[most_similar_cells.iloc[4].name])

    #getting max and min
    list_of_most_similar_cells = [number1, number2, number3, number4, number5]
    max_predicted_lifetime = max(list_of_most_similar_cells)
    min_predicted_lifetime = min(list_of_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")

    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)


/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -7.962522           -11.631570           -2.86283   
1            -7.962522           -11.825562           -2.86283   
2            -7.962522           -11.402131           -2.86283   
3            -7.962522           -10.099939           -2.86283   
4            -7.962522           -10.602896           -2.86283   
..                 ...                  ...                ...   
75           -7.962522            -7.948993           -2.86283   
76           -7.962522            -8.134920           -2.86283   
77           -7.962522            -8.484278           -2.86283   
78           -7.962522            -8.131572           -2.86283   
79           -7.962522            -7.650151           -2.86283   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901           -4.905592            -0.319238   
1            -4.709132           -4.905592            -1.148991   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.432684           -11.631570            -3.6381   
1            -9.432684           -11.825562            -3.6381   
2            -9.432684           -11.402131            -3.6381   
3            -9.432684           -10.099939            -3.6381   
4            -9.432684           -10.602896            -3.6381   
..                 ...                  ...                ...   
75           -9.432684            -7.948993            -3.6381   
76           -9.432684            -8.134920            -3.6381   
77           -9.432684            -8.484278            -3.6381   
78           -9.432684            -8.131572            -3.6381   
79           -9.432684            -7.650151            -3.6381   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901           -1.873386            -0.319238   
1            -4.709132           -1.873386            -1.148991   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.071859           -11.631570          -3.442255   
1            -9.071859           -11.825562          -3.442255   
2            -9.071859           -11.402131          -3.442255   
3            -9.071859           -10.099939          -3.442255   
4            -9.071859           -10.602896          -3.442255   
..                 ...                  ...                ...   
75           -9.071859            -7.948993          -3.442255   
76           -9.071859            -8.134920          -3.442255   
77           -9.071859            -8.484278          -3.442255   
78           -9.071859            -8.131572          -3.442255   
79           -9.071859            -7.650151          -3.442255   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901            -5.03744            -0.319238   
1            -4.709132            -5.03744            -1.148991   
2     

In [44]:
accuracy_list.count("No")

10

In [45]:
mean(range_list)

405.35

## FULL MODEL

In [46]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #slope_lin_fit_2_100
    df["cell_test_slope_lin_fit_2_100"] = pd.Series()
    df["cell_train_slope_lin_fit_2_100"] = pd.Series()
    #intercept_lin_fit_2_100
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series()
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #mean charge time
    df["cell_test_mean_charge_time"] = pd.Series()
    df["cell_train_mean_charge_time"] = pd.Series()  
    #minimum IR
    df["cell_test_minimum_IR"] = pd.Series()
    df["cell_train_minimum_IR"] = pd.Series()
    #diff_IR_100_2
    df["cell_test_diff_IR_100_2"] = pd.Series()
    df["cell_train_diff_IR_100_2"] = pd.Series()  
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_slope_lin_fit_2_100 = []
    cell_test_intercept_lin_fit_2_100 = []
    cell_test_dc2 = []
    cell_test_mean_charge_time = []
    cell_test_minimum_IR = []
    cell_test_diff_IR_100_2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_slope_lin_fit_2_100 = []
    cell_train_intercept_lin_fit_2_100 = []
    cell_train_dc2 = []
    cell_train_mean_charge_time = []
    cell_train_minimum_IR = []
    cell_train_diff_IR_100_2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_slope_lin_fit_2_100.append(x_test["slope_lin_fit_2_100"][i])
        cell_test_intercept_lin_fit_2_100.append(x_test["intercept_lin_fit_2_100"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_mean_charge_time.append(x_test["mean_charge_time"][i])
        cell_test_minimum_IR.append(x_test["minimum_IR"][i])
        cell_test_diff_IR_100_2.append(x_test["diff_IR_100_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_slope_lin_fit_2_100.append(x_train["slope_lin_fit_2_100"][a])
        cell_train_intercept_lin_fit_2_100.append(x_train["intercept_lin_fit_2_100"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_mean_charge_time.append(x_train["mean_charge_time"][a])
        cell_train_minimum_IR.append(x_train["minimum_IR"][a])
        cell_train_diff_IR_100_2.append(x_train["diff_IR_100_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_slope_lin_fit_2_100"] = pd.Series(cell_test_slope_lin_fit_2_100)
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series(cell_test_intercept_lin_fit_2_100)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_mean_charge_time"] = pd.Series(cell_test_mean_charge_time)
    df["cell_test_minimum_IR"] = pd.Series(cell_test_minimum_IR)
    df["cell_test_diff_IR_100_2"] = pd.Series(cell_test_diff_IR_100_2)


    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_slope_lin_fit_2_100"] = pd.Series(cell_train_slope_lin_fit_2_100)
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series(cell_train_intercept_lin_fit_2_100)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_mean_charge_time"] = pd.Series(cell_train_mean_charge_time)
    df["cell_train_minimum_IR"] = pd.Series(cell_train_minimum_IR)
    df["cell_train_diff_IR_100_2"] = pd.Series(cell_train_diff_IR_100_2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_slope_lin_fit_2_100"] - df["cell_train_slope_lin_fit_2_100"])**2
    d = (df["cell_test_intercept_lin_fit_2_100"] - df["cell_train_intercept_lin_fit_2_100"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_mean_charge_time"] - df["cell_train_mean_charge_time"])**2
    g = (df["cell_test_minimum_IR"] - df["cell_train_minimum_IR"])**2
    h = (df["cell_test_diff_IR_100_2"] - df["cell_train_diff_IR_100_2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f+g+h)

    #finding the most similar cells
    most_similar_cells = df.nsmallest(5, 'distance')
    number1 = int(y_train[most_similar_cells.iloc[0].name])
    number2 = int(y_train[most_similar_cells.iloc[1].name])
    number3 = int(y_train[most_similar_cells.iloc[2].name])
    number4 = int(y_train[most_similar_cells.iloc[3].name])
    number5 = int(y_train[most_similar_cells.iloc[4].name])

    #getting max and min
    list_of_most_similar_cells = [number1, number2, number3, number4, number5]
    max_predicted_lifetime = max(list_of_most_similar_cells)
    min_predicted_lifetime = min(list_of_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")

    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -7.962522           -11.631570           -2.86283   
1            -7.962522           -11.825562           -2.86283   
2            -7.962522           -11.402131           -2.86283   
3            -7.962522           -10.099939           -2.86283   
4            -7.962522           -10.602896           -2.86283   
..                 ...                  ...                ...   
75           -7.962522            -7.948993           -2.86283   
76           -7.962522            -8.134920           -2.86283   
77           -7.962522            -8.484278           -2.86283   
78           -7.962522            -8.131572           -2.86283   
79           -7.962522            -7.650151           -2.86283   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                       0.000022   
1            -4.709132                       0.000022   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy


Predicted lifetime range is from:  719 to  1074
Actual lifetime:  1063


    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.485617           -11.631570          -3.606255   
1            -9.485617           -11.825562          -3.606255   
2            -9.485617           -11.402131          -3.606255   
3            -9.485617           -10.099939          -3.606255   
4            -9.485617           -10.602896          -3.606255   
..                 ...                  ...                ...   
75           -9.485617            -7.948993          -3.606255   
76           -9.485617            -8.134920          -3.606255   
77           -9.485617            -8.484278          -3.606255   
78           -9.485617            -8.131572          -3.606255   
79           -9.485617            -7.650151          -3.606255   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                      -0.000013   
1            -4.709

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy


Predicted lifetime range is from:  719 to  880
Actual lifetime:  828


    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.196427           -11.631570           -3.52475   
1            -9.196427           -11.825562           -3.52475   
2            -9.196427           -11.402131           -3.52475   
3            -9.196427           -10.099939           -3.52475   
4            -9.196427           -10.602896           -3.52475   
..                 ...                  ...                ...   
75           -9.196427            -7.948993           -3.52475   
76           -9.196427            -8.134920           -3.52475   
77           -9.196427            -8.484278           -3.52475   
78           -9.196427            -8.131572           -3.52475   
79           -9.196427            -7.650151           -3.52475   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                   6.680532e-07   
1            -4.70913

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

In [47]:
accuracy_list.count("No")

7

In [48]:
mean(range_list)

352.5

# APPROACH 4 (10 most similar cells)

## VARIANCE

In [49]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    df["cell_test"] = pd.Series()
    df["cell_train"] = pd.Series()
    df["distance"] = pd.Series()
    cell_test = []
    cell_train = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test.append(x_test["variance_dQ_100_10"][i])
    for a in x_train.index:
        cell_train.append(x_train["variance_dQ_100_10"][a])
    df["cell_test"] = pd.Series(cell_test)
    df["cell_train"] = pd.Series(cell_train)
    df["distance"] = abs(df["cell_test"] - df["cell_train"])

    #finding the most similar cells
    most_similar_cells = df.nsmallest(10, 'distance')
    number1 = int(y_train[most_similar_cells.iloc[0].name])
    number2 = int(y_train[most_similar_cells.iloc[1].name])
    number3 = int(y_train[most_similar_cells.iloc[2].name])
    number4 = int(y_train[most_similar_cells.iloc[3].name])
    number5 = int(y_train[most_similar_cells.iloc[4].name])
    number6 = int(y_train[most_similar_cells.iloc[5].name])
    number7 = int(y_train[most_similar_cells.iloc[6].name])
    number8 = int(y_train[most_similar_cells.iloc[7].name])
    number9 = int(y_train[most_similar_cells.iloc[8].name])
    number10 = int(y_train[most_similar_cells.iloc[9].name])

    #getting max and min
    list_of_most_similar_cells = [number1, number2, number3, number4, number5, number6, number7, number8, number9, number10]
    max_predicted_lifetime = max(list_of_most_similar_cells)
    min_predicted_lifetime = min(list_of_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")

    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)

    cell_test  cell_train  distance
0   -7.962522  -11.631570  3.669048
1   -7.962522  -11.825562  3.863040
2   -7.962522  -11.402131  3.439609
3   -7.962522  -10.099939  2.137417
4   -7.962522  -10.602896  2.640374
..        ...         ...       ...
75  -7.962522   -7.948993  0.013529
76  -7.962522   -8.134920  0.172398
77  -7.962522   -8.484278  0.521756
78  -7.962522   -8.131572  0.169050
79  -7.962522   -7.650151  0.312371

[80 rows x 3 columns]
Predicted lifetime range is from:  444 to  617
Actual lifetime:  457


    cell_test  cell_train  distance
0   -8.233459  -11.631570  3.398111
1   -8.233459  -11.825562  3.592103
2   -8.233459  -11.402131  3.168672
3   -8.233459  -10.099939  1.866480
4   -8.233459  -10.602896  2.369437
..        ...         ...       ...
75  -8.233459   -7.948993  0.284466
76  -8.233459   -8.134920  0.098539
77  -8.233459   -8.484278  0.250819
78  -8.233459   -8.131572  0.101887
79  -8.233459   -7.650151  0.583308

[80 rows x 3 columns]
Predicted lifetime 

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/e

In [50]:
accuracy_list.count("No")

5

In [51]:
mean(range_list)

473.4

## DISCHARGE

In [52]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #skewness
    df["cell_test_skewness"] = pd.Series()
    df["cell_train_skewness"] = pd.Series()
    #kurtosis
    df["cell_test_kurtosis"] = pd.Series()
    df["cell_train_kurtosis"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #diff discharge capacity max 2
    df["cell_test_ddc2"] = pd.Series()
    df["cell_train_ddc2"] = pd.Series()
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_skewness = []
    cell_test_kurtosis = []
    cell_test_dc2 = []
    cell_test_ddc2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_skewness = []
    cell_train_kurtosis = []
    cell_train_dc2 = []
    cell_train_ddc2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_skewness.append(x_test["skewness_dQ_100_10"][i])
        cell_test_kurtosis.append(x_test["kurtosis_dQ_100_10"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_ddc2.append(x_test["diff_discharge_capacity_max_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_skewness.append(x_train["skewness_dQ_100_10"][a])
        cell_train_kurtosis.append(x_train["kurtosis_dQ_100_10"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_ddc2.append(x_train["diff_discharge_capacity_max_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_skewness"] = pd.Series(cell_test_skewness)
    df["cell_test_kurtosis"] = pd.Series(cell_test_kurtosis)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_ddc2"] = pd.Series(cell_test_ddc2)

    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_skewness"] = pd.Series(cell_train_skewness)
    df["cell_train_kurtosis"] = pd.Series(cell_train_kurtosis)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_ddc2"] = pd.Series(cell_train_ddc2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_skewness"] - df["cell_train_skewness"])**2
    d = (df["cell_test_kurtosis"] - df["cell_train_kurtosis"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_ddc2"] - df["cell_train_ddc2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f)

    #finding the most similar cells
    most_similar_cells = df.nsmallest(10, 'distance')
    number1 = int(y_train[most_similar_cells.iloc[0].name])
    number2 = int(y_train[most_similar_cells.iloc[1].name])
    number3 = int(y_train[most_similar_cells.iloc[2].name])
    number4 = int(y_train[most_similar_cells.iloc[3].name])
    number5 = int(y_train[most_similar_cells.iloc[4].name])
    number6 = int(y_train[most_similar_cells.iloc[5].name])
    number7 = int(y_train[most_similar_cells.iloc[6].name])
    number8 = int(y_train[most_similar_cells.iloc[7].name])
    number9 = int(y_train[most_similar_cells.iloc[8].name])
    number10 = int(y_train[most_similar_cells.iloc[9].name])

    #getting max and min
    list_of_most_similar_cells = [number1, number2, number3, number4, number5, number6, number7, number8, number9, number10]
    max_predicted_lifetime = max(list_of_most_similar_cells)
    min_predicted_lifetime = min(list_of_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")

    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -7.962522           -11.631570           -2.86283   
1            -7.962522           -11.825562           -2.86283   
2            -7.962522           -11.402131           -2.86283   
3            -7.962522           -10.099939           -2.86283   
4            -7.962522           -10.602896           -2.86283   
..                 ...                  ...                ...   
75           -7.962522            -7.948993           -2.86283   
76           -7.962522            -8.134920           -2.86283   
77           -7.962522            -8.484278           -2.86283   
78           -7.962522            -8.131572           -2.86283   
79           -7.962522            -7.650151           -2.86283   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901           -4.905592            -0.319238   
1            -4.709132           -4.905592            -1.148991   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.432684           -11.631570            -3.6381   
1            -9.432684           -11.825562            -3.6381   
2            -9.432684           -11.402131            -3.6381   
3            -9.432684           -10.099939            -3.6381   
4            -9.432684           -10.602896            -3.6381   
..                 ...                  ...                ...   
75           -9.432684            -7.948993            -3.6381   
76           -9.432684            -8.134920            -3.6381   
77           -9.432684            -8.484278            -3.6381   
78           -9.432684            -8.131572            -3.6381   
79           -9.432684            -7.650151            -3.6381   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901           -1.873386            -0.319238   
1            -4.709132           -1.873386            -1.148991   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

In [53]:
print(accuracy_list.count("No"))
print(mean(range_list))

4
588.05


## FULL MODEL

In [54]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #slope_lin_fit_2_100
    df["cell_test_slope_lin_fit_2_100"] = pd.Series()
    df["cell_train_slope_lin_fit_2_100"] = pd.Series()
    #intercept_lin_fit_2_100
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series()
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #mean charge time
    df["cell_test_mean_charge_time"] = pd.Series()
    df["cell_train_mean_charge_time"] = pd.Series()  
    #minimum IR
    df["cell_test_minimum_IR"] = pd.Series()
    df["cell_train_minimum_IR"] = pd.Series()
    #diff_IR_100_2
    df["cell_test_diff_IR_100_2"] = pd.Series()
    df["cell_train_diff_IR_100_2"] = pd.Series()  
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_slope_lin_fit_2_100 = []
    cell_test_intercept_lin_fit_2_100 = []
    cell_test_dc2 = []
    cell_test_mean_charge_time = []
    cell_test_minimum_IR = []
    cell_test_diff_IR_100_2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_slope_lin_fit_2_100 = []
    cell_train_intercept_lin_fit_2_100 = []
    cell_train_dc2 = []
    cell_train_mean_charge_time = []
    cell_train_minimum_IR = []
    cell_train_diff_IR_100_2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_slope_lin_fit_2_100.append(x_test["slope_lin_fit_2_100"][i])
        cell_test_intercept_lin_fit_2_100.append(x_test["intercept_lin_fit_2_100"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_mean_charge_time.append(x_test["mean_charge_time"][i])
        cell_test_minimum_IR.append(x_test["minimum_IR"][i])
        cell_test_diff_IR_100_2.append(x_test["diff_IR_100_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_slope_lin_fit_2_100.append(x_train["slope_lin_fit_2_100"][a])
        cell_train_intercept_lin_fit_2_100.append(x_train["intercept_lin_fit_2_100"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_mean_charge_time.append(x_train["mean_charge_time"][a])
        cell_train_minimum_IR.append(x_train["minimum_IR"][a])
        cell_train_diff_IR_100_2.append(x_train["diff_IR_100_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_slope_lin_fit_2_100"] = pd.Series(cell_test_slope_lin_fit_2_100)
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series(cell_test_intercept_lin_fit_2_100)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_mean_charge_time"] = pd.Series(cell_test_mean_charge_time)
    df["cell_test_minimum_IR"] = pd.Series(cell_test_minimum_IR)
    df["cell_test_diff_IR_100_2"] = pd.Series(cell_test_diff_IR_100_2)


    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_slope_lin_fit_2_100"] = pd.Series(cell_train_slope_lin_fit_2_100)
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series(cell_train_intercept_lin_fit_2_100)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_mean_charge_time"] = pd.Series(cell_train_mean_charge_time)
    df["cell_train_minimum_IR"] = pd.Series(cell_train_minimum_IR)
    df["cell_train_diff_IR_100_2"] = pd.Series(cell_train_diff_IR_100_2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_slope_lin_fit_2_100"] - df["cell_train_slope_lin_fit_2_100"])**2
    d = (df["cell_test_intercept_lin_fit_2_100"] - df["cell_train_intercept_lin_fit_2_100"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_mean_charge_time"] - df["cell_train_mean_charge_time"])**2
    g = (df["cell_test_minimum_IR"] - df["cell_train_minimum_IR"])**2
    h = (df["cell_test_diff_IR_100_2"] - df["cell_train_diff_IR_100_2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f+g+h)

    #finding the most similar cells
    most_similar_cells = df.nsmallest(10, 'distance')
    number1 = int(y_train[most_similar_cells.iloc[0].name])
    number2 = int(y_train[most_similar_cells.iloc[1].name])
    number3 = int(y_train[most_similar_cells.iloc[2].name])
    number4 = int(y_train[most_similar_cells.iloc[3].name])
    number5 = int(y_train[most_similar_cells.iloc[4].name])
    number6 = int(y_train[most_similar_cells.iloc[5].name])
    number7 = int(y_train[most_similar_cells.iloc[6].name])
    number8 = int(y_train[most_similar_cells.iloc[7].name])
    number9 = int(y_train[most_similar_cells.iloc[8].name])
    number10 = int(y_train[most_similar_cells.iloc[9].name])

    #getting max and min
    list_of_most_similar_cells = [number1, number2, number3, number4, number5, number6, number7, number8, number9, number10]
    max_predicted_lifetime = max(list_of_most_similar_cells)
    min_predicted_lifetime = min(list_of_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")

    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -7.962522           -11.631570           -2.86283   
1            -7.962522           -11.825562           -2.86283   
2            -7.962522           -11.402131           -2.86283   
3            -7.962522           -10.099939           -2.86283   
4            -7.962522           -10.602896           -2.86283   
..                 ...                  ...                ...   
75           -7.962522            -7.948993           -2.86283   
76           -7.962522            -8.134920           -2.86283   
77           -7.962522            -8.484278           -2.86283   
78           -7.962522            -8.131572           -2.86283   
79           -7.962522            -7.650151           -2.86283   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                       0.000022   
1            -4.709132                       0.000022   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -8.233459           -11.631570          -3.007304   
1            -8.233459           -11.825562          -3.007304   
2            -8.233459           -11.402131          -3.007304   
3            -8.233459           -10.099939          -3.007304   
4            -8.233459           -10.602896          -3.007304   
..                 ...                  ...                ...   
75           -8.233459            -7.948993          -3.007304   
76           -8.233459            -8.134920          -3.007304   
77           -8.233459            -8.484278          -3.007304   
78           -8.233459            -8.131572          -3.007304   
79           -8.233459            -7.650151          -3.007304   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                      -0.000007   
1            -4.709132                      -0.000007   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.485617           -11.631570          -3.606255   
1            -9.485617           -11.825562          -3.606255   
2            -9.485617           -11.402131          -3.606255   
3            -9.485617           -10.099939          -3.606255   
4            -9.485617           -10.602896          -3.606255   
..                 ...                  ...                ...   
75           -9.485617            -7.948993          -3.606255   
76           -9.485617            -8.134920          -3.606255   
77           -9.485617            -8.484278          -3.606255   
78           -9.485617            -8.131572          -3.606255   
79           -9.485617            -7.650151          -3.606255   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                      -0.000013   
1            -4.709132                      -0.000013   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0           -10.154227           -11.631570          -3.956446   
1           -10.154227           -11.825562          -3.956446   
2           -10.154227           -11.402131          -3.956446   
3           -10.154227           -10.099939          -3.956446   
4           -10.154227           -10.602896          -3.956446   
..                 ...                  ...                ...   
75          -10.154227            -7.948993          -3.956446   
76          -10.154227            -8.134920          -3.956446   
77          -10.154227            -8.484278          -3.956446   
78          -10.154227            -8.131572          -3.956446   
79          -10.154227            -7.650151          -3.956446   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                      -0.000003   
1            -4.709132                      -0.000003   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

In [55]:
print(accuracy_list.count("No"))
print(mean(range_list))

6
480.2
